# Benchmark: Implement Levenshtein term similarity matrix and fast SCM between corpora ([RaRe-Technologies/gensim PR #2016][#2016])

 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [1]:
!git rev-parse HEAD

20032f049fdfc7962e847d328dfe4696f60988c0


In [2]:
from copy import deepcopy
from datetime import timedelta
from itertools import product
import logging
from math import floor, ceil, log10
import pickle
from random import sample, seed, shuffle
from time import time

import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

from gensim.corpora import Dictionary
import gensim.downloader as api
from gensim.similarities.index import AnnoyIndexer
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import UniformTermSimilarityIndex
from gensim.similarities import LevenshteinSimilarityIndex
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.utils import simple_preprocess

RANDOM_SEED = 12345

logger = logging.getLogger(__name__)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)
pd.set_option('display.max_rows', None, 'display.max_seq_items', None)

In [3]:
"""Repeatedly run a benchmark callable given various configurations and
get a list of results.

Return a list of results of repeatedly running a benchmark callable.

Parameters
----------
benchmark : callable tuple -> dict
    A benchmark callable that accepts a configuration and returns results.
configurations : iterable of tuple
    An iterable of configurations that are used for calling the benchmark function.
results_filename : str
    A filename of a file that will be used to persistently store the results using
    pickle. If the file exists, then the function will load the stored results
    instead of calling the benchmark callable.

Returns
-------
iterable of tuple
    The return values of the individual invocations of the benchmark callable.

"""
def benchmark_results(benchmark, configurations, results_filename):
    try:
        with open(results_filename, "rb") as file:
            results = pickle.load(file)
    except IOError:
        configurations = list(configurations)
        shuffle(configurations)
        results = list(tqdm(
            (benchmark(configuration) for configuration in configurations),
            total=len(configurations), desc="benchmark"))
        with open(results_filename, "wb") as file:
            pickle.dump(results, file)
    return results

## Implement Levenshtein term similarity matrix

In Gensim PR [#1827][], we added a base implementation of the soft cosine measure (SCM). The base implementation would create term similarity matrices using a single complex procedure. In the Gensim PR [#2016][], we split the procedure into:

- **TermSimilarityIndex** builder classes that produce the $k$ most similar terms for a given term $t$ that are distinct from $t$ along with the term similarities, and
- the **SparseTermSimilarityMatrix** director class that constructs term similarity matrices and consumes term similarities produced by **TermSimilarityIndex** instances.

One of the benefits of this separation is that we can easily measure the speed at which a **TermSimilarityIndex** builder class produces term similarities and compare this speed with the speed at which the **SparseTermSimilarityMatrix** director class consumes term similarities. This allows us to see which of the classes are a bottleneck that slows down the construction of term similarity matrices.

In this notebook, we measure all the currently available builder and director classes. For the measurements, we use the [Google News word embeddings][word2vec-google-news-300] distributed with the C implementation of Word2Vec. From the word embeddings, we will derive a dictionary of 2.01M terms.

 [word2vec-google-news-300]: https://github.com/mmihaltz/word2vec-GoogleNews-vectors (word2vec-GoogleNews-vectors)
 [#1827]: https://github.com/RaRe-Technologies/gensim/pull/1827 (Implement Soft Cosine Measure - Pull Request #1827)
 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [4]:
full_model = api.load("word2vec-google-news-300")

try:
    full_dictionary = Dictionary.load("matrix_speed.dictionary")
except IOError:
    full_dictionary = Dictionary([[term] for term in full_model.vocab.keys()])
    full_dictionary.save("matrix_speed.dictionary")

### Director class benchmark
#### SparseTermSimilarityMatrix
First, we measure the speed at which the **SparseTermSimilarityMatrix** director class consumes term similarities.

In [5]:
def benchmark(configuration):
    dictionary, nonzero_limit, symmetric, repetition = configuration
    index = UniformTermSimilarityIndex(dictionary)
    
    start_time = time()
    matrix = SparseTermSimilarityMatrix(
        index, dictionary, nonzero_limit=nonzero_limit, symmetric=symmetric,
        dtype=np.float16).matrix
    end_time = time()
    
    duration = end_time - start_time
    return {
        "dictionary_size": len(dictionary),
        "nonzero_limit": nonzero_limit,
        "matrix_nonzero": matrix.nnz,
        "repetition": repetition,
        "symmetric": symmetric,
        "duration": duration, }

In [6]:
dictionary_sizes = [10**k for k in range(3, int(ceil(log10(len(full_dictionary)))))]
seed(RANDOM_SEED)
dictionaries = []
for size in tqdm(dictionary_sizes, desc="dictionaries"):
    dictionary = Dictionary([sample(list(full_dictionary.values()), size)])
    dictionaries.append(dictionary)
dictionaries.append(full_dictionary)
nonzero_limits = [1, 10, 100]
symmetry = (True, False)
repetitions = range(10)

configurations = product(dictionaries, nonzero_limits, symmetry, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.director_results")

dictionaries: 100%|██████████| 4/4 [00:14<00:00,  3.61s/it]


The following tables show how long it takes to construct a term similarity matrix (the **duration** column), how many nonzero elements there are in the matrix (the **matrix_nonzero** column) and the mean term similarity consumption speed (the **consumption_speed** column) as we vary the dictionary size (the **dictionary_size** column) the maximum number of nonzero elements outside the diagonal in every column of the matrix (the **nonzero_limit** column), and the matrix symmetry constraint (the **symmetric** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

We can see that the symmetry constraint severely limits the number of nonzero elements in the resulting matrix. This in turn increases the consumption speed, since we end up throwing away most of the elements that we consume. The effects of the dictionary size on the mean term similarity consumption speed are minor.

In [7]:
df = pd.DataFrame(results)
df["consumption_speed"] = df.dictionary_size * df.nonzero_limit / df.duration
df = df.groupby(["dictionary_size", "nonzero_limit", "symmetric"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["matrix_nonzero"] = [int(nonzero) for nonzero in df["matrix_nonzero"]]
    df["consumption_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["consumption_speed"]]
    return df

In [8]:
display(df.mean()).loc[
    [10000, len(full_dictionary)], :, :].loc[
    :, ["duration", "matrix_nonzero", "consumption_speed"]]

duration  matrix_nonzero  \
dictionary_size nonzero_limit symmetric                                   
10000           1             False     00:00:00.286091           20000   
                              True      00:00:00.166287           10002   
                10            False     00:00:01.573833          110000   
                              True      00:00:00.640328           10118   
                100           False     00:00:14.662728         1010000   
                              True      00:00:05.233251           20198   
2010000         1             False     00:01:02.938585         4020000   
                              True      00:00:35.977733         2010002   
                10            False     00:05:53.410117        22110000   
                              True      00:02:07.940066         2010118   
                100           False     01:06:08.946648       203010000   
                              True      00:17:06.064969         2020198   

                                              consumption_speed  
dictionary_size nonzero_limit symmetric                          
10000           1             False       34.95 Kword pairs / s  
                              True        60.15 Kword pairs / s  
                10            False       63.54 Kword pairs / s  
                              True       156.20 Kword pairs / s  
                100           False       68.20 Kword pairs / s  
                              True       191.09 Kword pairs / s  
2010000         1             False       31.94 Kword pairs / s  
                              True        55.87 Kword pairs / s  
                10            False       56.88 Kword pairs / s  
                              True       157.11 Kword pairs / s  
                100           False       50.65 Kword pairs / s  
                              True       195.91 Kword pairs / s

In [9]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [10000, len(full_dictionary)], :, :].loc[
    :, ["duration", "matrix_nonzero", "consumption_speed"]]

duration  matrix_nonzero  \
dictionary_size nonzero_limit symmetric                                   
10000           1             False     00:00:00.001519               0   
                              True      00:00:00.002255               0   
                10            False     00:00:00.013232               0   
                              True      00:00:00.009424               0   
                100           False     00:00:00.101245               0   
                              True      00:00:00.021103               0   
2010000         1             False     00:00:00.205360               0   
                              True      00:00:00.091344               0   
                10            False     00:00:01.252888               0   
                              True      00:00:00.302513               0   
                100           False     00:00:54.806356               0   
                              True      00:00:09.599669               0   

                                            consumption_speed  
dictionary_size nonzero_limit symmetric                        
10000           1             False      0.19 Kword pairs / s  
                              True       0.82 Kword pairs / s  
                10            False      0.53 Kword pairs / s  
                              True       2.27 Kword pairs / s  
                100           False      0.47 Kword pairs / s  
                              True       0.77 Kword pairs / s  
2010000         1             False      0.10 Kword pairs / s  
                              True       0.14 Kword pairs / s  
                10            False      0.20 Kword pairs / s  
                              True       0.37 Kword pairs / s  
                100           False      0.68 Kword pairs / s  
                              True       1.82 Kword pairs / s

### Builder class benchmark
#### UniformTermSimilarityIndex
First, we measure the speed at which the **UniformTermSimilarityIndex** builder class produces term similarities. **UniformTermSimilarityIndex** is a dummy class that just generates a sequence of constants. It produces much more term similarities per second than the **SparseTermSimilarityMatrix** is capable of consuming and its results will serve as an upper limit.

In [10]:
def benchmark(configuration):
    dictionary, nonzero_limit, repetition = configuration
    
    start_time = time()
    index = UniformTermSimilarityIndex(dictionary)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in dictionary.values():
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "nonzero_limit": nonzero_limit,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [11]:
nonzero_limits = [1, 10, 100, 1000]

configurations = product(dictionaries, nonzero_limits, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.uniform")

The following tables show how long it takes to retrieve the most similar terms for all terms in a dictionary (the **production_duration** column) and the mean term similarity production speed (the **production_speed** column) as we vary the dictionary size (the **dictionary_size** column), and the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **production_speed** is proportional to **nonzero_limit**.

In [12]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size ** 2 / df.production_duration
df["production_speed"] = df.dictionary_size * df.nonzero_limit / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["production_speed"]]
    return df

In [13]:
display(df.mean()).loc[
    [1000, len(full_dictionary)], :, :].loc[
    :, ["production_duration", "production_speed"]]

production_duration         production_speed
dictionary_size nonzero_limit                                             
1000            1                 00:00:00.003828   261.66 Kword pairs / s
                10                00:00:00.009975  1002.80 Kword pairs / s
                100               00:00:00.073020  1372.84 Kword pairs / s
                1000              00:00:00.727086  1375.54 Kword pairs / s
2010000         1                 00:00:08.315807   241.71 Kword pairs / s
                10                00:00:21.027485   955.90 Kword pairs / s
                100               00:02:24.223933  1393.70 Kword pairs / s
                1000              00:23:57.702287  1398.09 Kword pairs / s

In [14]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, len(full_dictionary)], :, :].loc[
    :, ["production_duration", "production_speed"]]

production_duration       production_speed
dictionary_size nonzero_limit                                           
1000            1                 00:00:00.000163  10.67 Kword pairs / s
                10                00:00:00.000174  17.08 Kword pairs / s
                100               00:00:00.004030  67.59 Kword pairs / s
                1000              00:00:00.009082  16.98 Kword pairs / s
2010000         1                 00:00:00.023885   0.70 Kword pairs / s
                10                00:00:00.054096   2.46 Kword pairs / s
                100               00:00:00.763313   7.42 Kword pairs / s
                1000              00:00:06.162980   5.99 Kword pairs / s

#### LevenshteinSimilarityIndex
Next, we measure the speed at which the **LevenshteinSimilarityIndex** builder class produces term similarities. **LevenshteinSimilarityIndex** is currently just a naïve implementation that produces much fewer term similarities per second than the **SparseTermSimilarityMatrix** class is capable of consuming.

In [15]:
def benchmark(configuration):
    dictionary, nonzero_limit, workers, query_terms, repetition = configuration
    
    start_time = time()
    index = LevenshteinSimilarityIndex(dictionary, workers=workers)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in query_terms:
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "mean_query_term_length": np.mean([len(term) for term in query_terms]),
        "nonzero_limit": nonzero_limit,
        "workers": workers,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [16]:
nonzero_limits = [1, 10, 100]
workers = [2**k for k in range(5)]
seed(RANDOM_SEED)
min_dictionary = sorted((len(dictionary), dictionary) for dictionary in dictionaries)[0][1]
query_terms = sample(list(min_dictionary.values()), 10)

configurations = product(dictionaries, nonzero_limits, workers, [query_terms], repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.levenshtein")

The following tables show how long it takes to retrieve the most similar terms for ten randomly sampled terms from a dictionary (the **production_duration** column), the mean term similarity production speed (the **production_speed** column) and the mean term similarity processing speed (the **processing_speed** column) as we vary the dictionary size (the **dictionary_size** column), the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column), and the number of worker processes (the **workers** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **production_speed** is proportional to **nonzero_limit / dictionary_size**. Notably, **production_speed** does not increase with **workers**.

In [17]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size * len(query_terms) / df.production_duration
df["production_speed"] = df.nonzero_limit * len(query_terms) / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit", "workers"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f word pairs / s" % speed for speed in df["production_speed"]]
    return df

In [18]:
display(df.mean()).loc[
    [1000, 1000000, len(full_dictionary)], :, [1, 16]].loc[
    :, ["production_duration", "production_speed", "processing_speed"]]

production_duration  \
dictionary_size nonzero_limit workers                       
1000            1             1           00:00:00.088804   
                              16          00:00:00.110414   
                10            1           00:00:00.088173   
                              16          00:00:00.103538   
                100           1           00:00:00.084138   
                              16          00:00:00.098996   
1000000         1             1           00:00:58.555345   
                              16          00:01:00.098409   
                10            1           00:00:58.418561   
                              16          00:00:59.656511   
                100           1           00:00:59.695258   
                              16          00:00:59.806840   
2010000         1             1           00:01:58.215667   
                              16          00:01:59.468716   
                10            1           00:01:58.497900   
                              16          00:01:58.750654   
                100           1           00:01:57.635140   
                              16          00:02:00.499245   

                                              production_speed  \
dictionary_size nonzero_limit workers                            
1000            1             1          112.66 word pairs / s   
                              16          90.62 word pairs / s   
                10            1         1135.12 word pairs / s   
                              16         966.10 word pairs / s   
                100           1        11896.50 word pairs / s   
                              16       10104.65 word pairs / s   
1000000         1             1            0.17 word pairs / s   
                              16           0.17 word pairs / s   
                10            1            1.71 word pairs / s   
                              16           1.68 word pairs / s   
                100           1           16.76 word pairs / s   
                              16          16.73 word pairs / s   
2010000         1             1            0.08 word pairs / s   
                              16           0.08 word pairs / s   
                10            1            0.84 word pairs / s   
                              16           0.84 word pairs / s   
                100           1            8.51 word pairs / s   
                              16           8.30 word pairs / s   

                                             processing_speed  
dictionary_size nonzero_limit workers                          
1000            1             1        112.66 Kword pairs / s  
                              16        90.62 Kword pairs / s  
                10            1        113.51 Kword pairs / s  
                              16        96.61 Kword pairs / s  
                100           1        118.96 Kword pairs / s  
                              16       101.05 Kword pairs / s  
1000000         1             1        170.86 Kword pairs / s  
                              16       166.47 Kword pairs / s  
                10            1        171.23 Kword pairs / s  
                              16       167.66 Kword pairs / s  
                100           1        167.59 Kword pairs / s  
                              16       167.32 Kword pairs / s  
2010000         1             1        170.12 Kword pairs / s  
                              16       168.32 Kword pairs / s  
                10            1        169.68 Kword pairs / s  
                              16       169.38 Kword pairs / s  
                100           1        170.96 Kword pairs / s  
                              16       166.90 Kword pairs / s

In [19]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 1000000, len(full_dictionary)], :, [1, 16]].loc[
    :, ["production_duration", "production_speed", "processing_speed"]]

production_duration  \
dictionary_size nonzero_limit workers                       
1000            1             1           00:00:00.002072   
                              16          00:00:00.002730   
                10            1           00:00:00.002696   
                              16          00:00:00.001824   
                100           1           00:00:00.002675   
                              16          00:00:00.001845   
1000000         1             1           00:00:01.315360   
                              16          00:00:01.375036   
                10            1           00:00:01.088038   
                              16          00:00:00.909982   
                100           1           00:00:01.286779   
                              16          00:00:01.672433   
2010000         1             1           00:00:02.940157   
                              16          00:00:02.630008   
                10            1           00:00:02.306872   
                              16          00:00:03.301690   
                100           1           00:00:02.922025   
                              16          00:00:03.075475   

                                            production_speed  \
dictionary_size nonzero_limit workers                          
1000            1             1          2.65 word pairs / s   
                              16         2.25 word pairs / s   
                10            1         36.02 word pairs / s   
                              16        17.08 word pairs / s   
                100           1        394.92 word pairs / s   
                              16       191.00 word pairs / s   
1000000         1             1          0.00 word pairs / s   
                              16         0.00 word pairs / s   
                10            1          0.03 word pairs / s   
                              16         0.03 word pairs / s   
                100           1          0.37 word pairs / s   
                              16         0.48 word pairs / s   
2010000         1             1          0.00 word pairs / s   
                              16         0.00 word pairs / s   
                10            1          0.02 word pairs / s   
                              16         0.02 word pairs / s   
                100           1          0.22 word pairs / s   
                              16         0.21 word pairs / s   

                                           processing_speed  
dictionary_size nonzero_limit workers                        
1000            1             1        2.65 Kword pairs / s  
                              16       2.25 Kword pairs / s  
                10            1        3.60 Kword pairs / s  
                              16       1.71 Kword pairs / s  
                100           1        3.95 Kword pairs / s  
                              16       1.91 Kword pairs / s  
1000000         1             1        3.87 Kword pairs / s  
                              16       3.80 Kword pairs / s  
                10            1        3.21 Kword pairs / s  
                              16       2.56 Kword pairs / s  
                100           1        3.67 Kword pairs / s  
                              16       4.76 Kword pairs / s  
2010000         1             1        4.29 Kword pairs / s  
                              16       3.62 Kword pairs / s  
                10            1        3.31 Kword pairs / s  
                              16       4.62 Kword pairs / s  
                100           1        4.34 Kword pairs / s  
                              16       4.28 Kword pairs / s

#### WordEmbeddingSimilarityIndex
Lastly, we measure the speed at which the **WordEmbeddingSimilarityIndex** builder class constructs an instance and produces term similarities. Gensim currently supports slow and precise nearest neighbor search, and also approximate nearest neighbor search using [ANNOY][]. We evaluate both options.

 [ANNOY]: https://github.com/spotify/annoy (Approximate Nearest Neighbors in C++/Python optimized for memory usage and loading/saving to disk)

In [20]:
def benchmark(configuration):
    (model, dictionary), nonzero_limit, annoy_n_trees, query_terms, repetition = configuration
    use_annoy = annoy_n_trees > 0
    model.init_sims()
    
    start_time = time()
    if use_annoy:
        annoy = AnnoyIndexer(model, annoy_n_trees)
        kwargs = {"indexer": annoy}
    else:
        kwargs = {}
    index = WordEmbeddingSimilarityIndex(model, kwargs=kwargs)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in query_terms:
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "mean_query_term_length": np.mean([len(term) for term in query_terms]),
        "nonzero_limit": nonzero_limit,
        "use_annoy": use_annoy,
        "annoy_n_trees": annoy_n_trees,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [21]:
models = []
for dictionary in tqdm(dictionaries, desc="models"):
    if dictionary == full_dictionary:
        models.append(full_model)
        continue
    model = full_model.__class__(full_model.vector_size)
    model.vocab = {word: deepcopy(full_model.vocab[word]) for word in dictionary.values()}
    model.index2entity = []
    vector_indices = []
    for index, word in enumerate(full_model.index2entity):
        if word in model.vocab.keys():
            model.index2entity.append(word)
            model.vocab[word].index = len(vector_indices)
            vector_indices.append(index)
    model.vectors = full_model.vectors[vector_indices]
    models.append(model)
annoy_n_trees = [0] + [10**k for k in range(3)]
seed(RANDOM_SEED)
query_terms = sample(list(min_dictionary.values()), 1000)

configurations = product(zip(models, dictionaries), nonzero_limits, annoy_n_trees, [query_terms], repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.wordembeddings")

models: 100%|██████████| 5/5 [00:42<00:00,  8.44s/it]


The following tables show how long it takes to construct an ANNOY index and the builder class instance (the **constructor_duration** column), how long it takes to retrieve the most similar terms for 1,000 randomly sampled terms from a dictionary (the **production_duration** column), the mean term similarity production speed (the **production_speed** column) and the mean term similarity processing speed (the **processing_speed** column) as we vary the dictionary size (the **dictionary_size** column), the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column), and the number of constructed ANNOY trees (the **annoy_n_trees** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

If we do not use ANNOY (**annoy_n_trees**${}=0$), then **production_speed** is proportional to **nonzero_limit / dictionary_size**. 
If we do use ANNOY (**annoy_n_trees**${}>0$), then **production_speed** is proportional to **nonzero_limit / (annoy_n_trees)**${}^{1/2}$.

In [22]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size * len(query_terms) / df.production_duration
df["production_speed"] = df.nonzero_limit * len(query_terms) / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit", "annoy_n_trees"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["production_speed"]]
    return df

In [23]:
display(df.mean()).loc[
    [1000000, len(full_dictionary)], [1, 100], [0, 1, 100]].loc[
    :, ["constructor_duration", "production_duration", "production_speed", "processing_speed"]]

constructor_duration  \
dictionary_size nonzero_limit annoy_n_trees                        
1000000         1             0                  00:00:00.000016   
                              1                  00:00:29.243768   
                              100                00:06:18.394023   
                100           0                  00:00:00.000014   
                              1                  00:00:29.327988   
                              100                00:06:17.906254   
2010000         1             0                  00:00:00.000013   
                              1                  00:01:30.093113   
                              100                00:23:21.211156   
                100           0                  00:00:00.000012   
                              1                  00:01:30.555084   
                              100                00:22:10.017924   

                                            production_duration  \
dictionary_size nonzero_limit annoy_n_trees                       
1000000         1             0                 00:00:19.814856   
                              1                 00:00:00.086151   
                              100               00:00:00.145153   
                100           0                 00:00:21.404975   
                              1                 00:00:00.148678   
                              100               00:00:01.267254   
2010000         1             0                 00:01:55.708445   
                              1                 00:00:00.169142   
                              100               00:00:00.317731   
                100           0                 00:02:12.106273   
                              1                 00:00:00.271253   
                              100               00:00:02.856382   

                                                   production_speed  \
dictionary_size nonzero_limit annoy_n_trees                           
1000000         1             0                0.05 Kword pairs / s   
                              1               11.61 Kword pairs / s   
                              100              6.89 Kword pairs / s   
                100           0                4.67 Kword pairs / s   
                              1              672.60 Kword pairs / s   
                              100             78.91 Kword pairs / s   
2010000         1             0                0.01 Kword pairs / s   
                              1                5.91 Kword pairs / s   
                              100              3.15 Kword pairs / s   
                100           0                0.76 Kword pairs / s   
                              1              368.69 Kword pairs / s   
                              100             35.01 Kword pairs / s   

                                                        processing_speed  
dictionary_size nonzero_limit annoy_n_trees                               
1000000         1             0                 50467.35 Kword pairs / s  
                              1              11607994.56 Kword pairs / s  
                              100             6889489.33 Kword pairs / s  
                100           0                 46718.15 Kword pairs / s  
                              1               6725972.71 Kword pairs / s  
                              100              789115.67 Kword pairs / s  
2010000         1             0                 17371.28 Kword pairs / s  
                              1              11883667.16 Kword pairs / s  
                              100             6341482.88 Kword pairs / s  
                100           0                 15215.26 Kword pairs / s  
                              1               7410628.89 Kword pairs / s  
                              100              703700.97 Kword pairs / s

In [24]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000000, len(full_dictionary)], [1, 100], [0, 1, 100]].loc[
    :, ["constructor_duration", "production_duration", "production_speed", "processing_speed"]]

constructor_duration  \
dictionary_size nonzero_limit annoy_n_trees                        
1000000         1             0                  00:00:00.000007   
                              1                  00:00:00.037389   
                              100                00:00:00.778346   
                100           0                         00:00:00   
                              1                  00:00:00.230572   
                              100                00:00:00.365938   
2010000         1             0                         00:00:00   
                              1                  00:00:00.054403   
                              100                00:02:36.227334   
                100           0                  00:00:00.000001   
                              1                  00:00:00.961168   
                              100                00:00:05.000325   

                                            production_duration  \
dictionary_size nonzero_limit annoy_n_trees                       
1000000         1             0                 00:00:00.038433   
                              1                 00:00:00.000601   
                              100               00:00:00.000842   
                100           0                 00:00:00.019706   
                              1                 00:00:00.000249   
                              100               00:00:00.004160   
2010000         1             0                 00:00:00.165553   
                              1                 00:00:00.000622   
                              100               00:00:00.017605   
                100           0                 00:00:00.546129   
                              1                 00:00:00.002506   
                              100               00:00:00.013226   

                                                 production_speed  \
dictionary_size nonzero_limit annoy_n_trees                         
1000000         1             0              0.00 Kword pairs / s   
                              1              0.08 Kword pairs / s   
                              100            0.04 Kword pairs / s   
                100           0              0.00 Kword pairs / s   
                              1              1.13 Kword pairs / s   
                              100            0.26 Kword pairs / s   
2010000         1             0              0.00 Kword pairs / s   
                              1              0.02 Kword pairs / s   
                              100            0.15 Kword pairs / s   
                100           0              0.00 Kword pairs / s   
                              1              3.35 Kword pairs / s   
                              100            0.16 Kword pairs / s   

                                                      processing_speed  
dictionary_size nonzero_limit annoy_n_trees                             
1000000         1             0                  97.76 Kword pairs / s  
                              1               79750.61 Kword pairs / s  
                              100             39420.33 Kword pairs / s  
                100           0                  43.04 Kword pairs / s  
                              1               11255.12 Kword pairs / s  
                              100              2582.72 Kword pairs / s  
2010000         1             0                  24.79 Kword pairs / s  
                              1               43763.82 Kword pairs / s  
                              100            308488.25 Kword pairs / s  
                100           0                  62.80 Kword pairs / s  
                              1               67336.24 Kword pairs / s  
                              100              3258.56 Kword pairs / s

## Implement fast SCM between corpora

In Gensim PR [#1827][], we added a base implementation of the soft cosine measure (SCM). The base implementation would compute SCM between single documents using the **softcossim** function. In the Gensim PR [#2016][], we intruduced the **SparseTermSimilarityMatrix.inner_product** method, which computes SCM not only between single documents, but also between a document and a corpus, and between two corpora.

For the measurements, we use the [Google News word embeddings][word2vec-google-news-300] distributed with the C implementation of Word2Vec. From the word embeddings, we will derive a dictionary of 2.01m terms. As a corpus, we will use a random sample of 100K articles from the 4.92m English [Wikipedia articles][enwiki].

 [word2vec-google-news-300]: https://github.com/mmihaltz/word2vec-GoogleNews-vectors (word2vec-GoogleNews-vectors)
 [enwiki]: https://github.com/RaRe-Technologies/gensim-data/releases/tag/wiki-english-20171001 (wiki-english-20171001)
 [#1827]: https://github.com/RaRe-Technologies/gensim/pull/1827 (Implement Soft Cosine Measure - Pull Request #1827)
 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [25]:
full_model = api.load("word2vec-google-news-300")

try:
    with open("matrix_speed.corpus", "rb") as file:
        full_corpus = pickle.load(file)        
except IOError:
    original_corpus = list(tqdm(api.load("wiki-english-20171001"), desc="original_corpus", total=4924894))
    seed(RANDOM_SEED)
    full_corpus = [
        simple_preprocess(u'\n'.join(article["section_texts"]))
        for article in tqdm(sample(original_corpus, 10**5), desc="full_corpus", total=10**5)]
    del original_corpus
    with open("matrix_speed.corpus", "wb") as file:
        pickle.dump(full_corpus, file)

try:
    full_dictionary = Dictionary.load("matrix_speed.dictionary")
except IOError:
    full_dictionary = Dictionary([[term] for term in full_model.vocab.keys()])
    full_dictionary.save("matrix_speed.dictionary")

### SCM between two documents
First, we measure the speed at which the **inner_product** method produces term similarities between single documents.

In [26]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    corpus = [vec for vec in corpus if len(vec) > 0]
    
    start_time = time()
    for vec1 in corpus:
        for vec2 in corpus:
            matrix.inner_product(vec1, vec2, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [27]:
seed(RANDOM_SEED)
dictionary_sizes = [1000, 10000, 100000]
dictionaries = []
for size in tqdm(dictionary_sizes, desc="dictionaries"):
    dictionary = Dictionary([sample(list(full_dictionary.values()), size)])
    dictionaries.append(dictionary)
min_dictionary = sorted((len(dictionary), dictionary) for dictionary in dictionaries)[0][1]

corpus_sizes = [100, 1000]
corpora = []
for size in tqdm(corpus_sizes, desc="corpora"):
    corpus = sample(full_corpus, size)
    corpora.append(corpus)

models = []
for dictionary in tqdm(dictionaries, desc="models"):
    if dictionary == full_dictionary:
        models.append(full_model)
        continue
    model = full_model.__class__(full_model.vector_size)
    model.vocab = {word: deepcopy(full_model.vocab[word]) for word in dictionary.values()}
    model.index2entity = []
    vector_indices = []
    for index, word in enumerate(full_model.index2entity):
        if word in model.vocab.keys():
            model.index2entity.append(word)
            model.vocab[word].index = len(vector_indices)
            vector_indices.append(index)
    model.vectors = full_model.vectors[vector_indices]
    models.append(model)

nonzero_limits = [1, 10, 100]
matrices = []
for (model, dictionary), nonzero_limit in tqdm(
        list(product(zip(models, dictionaries), nonzero_limits)), desc="matrices"):
    index = WordEmbeddingSimilarityIndex(model)
    matrix = SparseTermSimilarityMatrix(index, dictionary, nonzero_limit=nonzero_limit)
    matrices.append((matrix, dictionary, nonzero_limit))

normalization = (True, False)
repetitions = range(10)

matrices: 100%|██████████| 9/9 [27:11<00:00, 181.33s/it]


In [28]:
configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.doc_doc")

The following tables show how long it takes to compute the **inner_product** method between all document vectors in a corpus (the **duration** column), how many nonzero document vectors there are in a corpus (the **corpus_nonzero** column), how many nonzero elements there are in a term similarity matrix (the **matrix_nonzero** column) and the mean document similarity production speed (the **speed** column) as we vary the dictionary size (the **dictionary_size** column), the size of the corpus (the **corpus_size** column), the maximum number of nonzero elements in a single column of the matrix (the **nonzero_limit** column), and the matrix symmetry constraint (the **symmetric** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **speed** is proportional to the square of the number of unique terms shared by the two document vectors. In our scenario as well as the standard IR scenario, this means **speed** is constant. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [29]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [30]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000771   
                                          True       00:00:00.000839   
                            100           False      00:00:00.000754   
                                          True       00:00:00.000818   
                1000        1             False      00:00:00.122427   
                                          True       00:00:00.135378   
                            100           False      00:00:00.123519   
                                          True       00:00:00.135408   
100000          100         1             False      00:00:05.155703   
                                          True       00:00:05.651955   
                            100           False      00:00:05.241041   
                                          True       00:00:05.748897   
                1000        1             False      00:07:32.158992   
                                          True       00:08:13.536636   
                            100           False      00:07:42.383915   
                                          True       00:08:25.955335   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  1.0   
                                          True                   1.0   
                            100           False                  1.0   
                                          True                   1.0   
                1000        1             False                 14.0   
                                          True                  14.0   
                            100           False                 14.0   
                                          True                  14.0   
100000          100         1             False                 89.0   
                                          True                  89.0   
                            100           False                 89.0   
                                          True                  89.0   
                1000        1             False                830.0   
                                          True                 830.0   
                            100           False                830.0   
                                          True                 830.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
                1000        1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
100000          100         1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   
                1000        1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   

                                                                    speed  
dictionary_size corpus_size nonzero_limit normalized                       
1000            1

In [31]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000041   
                                          True       00:00:00.000063   
                            100           False      00:00:00.000036   
                                          True       00:00:00.000054   
                1000        1             False      00:00:00.001614   
                                          True       00:00:00.003312   
                            100           False      00:00:00.001759   
                                          True       00:00:00.002503   
100000          100         1             False      00:00:00.051677   
                                          True       00:00:00.066897   
                            100           False      00:00:00.044883   
                                          True       00:00:00.049135   
                1000        1             False      00:00:01.874406   
                                          True       00:00:01.320134   
                            100           False      00:00:03.193139   
                                          True       00:00:01.807325   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                                    speed  
dictionary_size corpus_size nonzero_limit normalized                       
1000            1

### SCM between a document and a corpus
Next, we measure the speed at which the **inner_product** method produces term similarities between documents and a corpus.

In [32]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    corpus = [vec for vec in corpus if len(vec) > 0]
    
    start_time = time()
    for vec in corpus:
        matrix.inner_product(vec, corpus, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [33]:
configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.doc_corpus")

The **speed** is inversely proportional to **matrix_nonzero**. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [34]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [35]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.001281   
                                          True       00:00:00.003343   
                            100           False      00:00:00.003151   
                                          True       00:00:00.006315   
                1000        1             False      00:00:00.012436   
                                          True       00:00:00.045385   
                            100           False      00:00:00.022614   
                                          True       00:00:00.073368   
100000          100         1             False      00:00:00.275588   
                                          True       00:00:00.832673   
                            100           False      00:00:40.486930   
                                          True       00:02:01.491995   
                1000        1             False      00:00:04.079616   
                                          True       00:00:09.576150   
                            100           False      00:06:12.521743   
                                          True       00:17:56.378400   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  1.0   
                                          True                   1.0   
                            100           False                  1.0   
                                          True                   1.0   
                1000        1             False                 14.0   
                                          True                  14.0   
                            100           False                 14.0   
                                          True                  14.0   
100000          100         1             False                 89.0   
                                          True                  89.0   
                            100           False                 89.0   
                                          True                  89.0   
                1000        1             False                830.0   
                                          True                 830.0   
                            100           False                830.0   
                                          True                 830.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
                1000        1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
100000          100         1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   
                1000        1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   

                                                                      speed  
dictionary_size corpus_size nonzero_limit normalized                         
1000         

In [36]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000161   
                                          True       00:00:00.000304   
                            100           False      00:00:00.000456   
                                          True       00:00:00.000811   
                1000        1             False      00:00:00.000088   
                                          True       00:00:00.000521   
                            100           False      00:00:00.000231   
                                          True       00:00:00.000915   
100000          100         1             False      00:00:00.004586   
                                          True       00:00:00.008754   
                            100           False      00:00:00.054110   
                                          True       00:00:00.192001   
                1000        1             False      00:00:00.020248   
                                          True       00:00:00.021712   
                            100           False      00:00:14.605401   
                                          True       00:01:18.415992   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                                    speed  
dictionary_size corpus_size nonzero_limit normalized                       
1000            1

### SCM between two corpora
Lastly, we measure the speed at which the **inner_product** method produces term similarities between entire corpora.

In [37]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    corpus = [vec for vec in corpus if len(vec) > 0]
    
    start_time = time()
    matrix.inner_product(corpus, corpus, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [38]:
configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.corpus_corpus")

The **speed** is inversely proportional to **matrix_nonzero**. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [39]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [43]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.001247   
                                          True       00:00:00.003183   
                            100           False      00:00:00.002251   
                                          True       00:00:00.005614   
                1000        1             False      00:00:00.001136   
                                          True       00:00:00.004008   
                            100           False      00:00:00.002621   
                                          True       00:00:00.006765   
100000          100         1             False      00:00:00.004793   
                                          True       00:00:00.011828   
                            100           False      00:00:00.350742   
                                          True       00:00:01.122381   
                1000        1             False      00:00:00.013180   
                                          True       00:00:00.020674   
                            100           False      00:00:00.386497   
                                          True       00:00:01.150718   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  1.0   
                                          True                   1.0   
                            100           False                  1.0   
                                          True                   1.0   
                1000        1             False                 14.0   
                                          True                  14.0   
                            100           False                 14.0   
                                          True                  14.0   
100000          100         1             False                 89.0   
                                          True                  89.0   
                            100           False                 89.0   
                                          True                  89.0   
                1000        1             False                830.0   
                                          True                 830.0   
                            100           False                830.0   
                                          True                 830.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
                1000        1             False               1672.0   
                                          True                1672.0   
                            100           False              80448.0   
                                          True               80448.0   
100000          100         1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   
                1000        1             False             160806.0   
                                          True              160806.0   
                            100           False            7984456.0   
                                          True             7984456.0   

                                                                        speed  
dictionary_size corpus_size nonzero_limit normalized                           
1000     

In [41]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000252   
                                          True       00:00:00.000272   
                            100           False      00:00:00.000507   
                                          True       00:00:00.000400   
                1000        1             False      00:00:00.000046   
                                          True       00:00:00.000065   
                            100           False      00:00:00.000075   
                                          True       00:00:00.000187   
100000          100         1             False      00:00:00.000436   
                                          True       00:00:00.000618   
                            100           False      00:00:00.012132   
                                          True       00:00:00.154194   
                1000        1             False      00:00:00.000687   
                                          True       00:00:00.000325   
                            100           False      00:00:00.041810   
                                          True       00:00:00.130642   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                                       speed  
dictionary_size corpus_size nonzero_limit normalized                          
1000       